### Compare Listing 
<li>a: vector uniform</li>
<li>b: greedy</li>
<li>c: e - greedy</li>
<li>d: decay e - greedy</li>
<li>e: Linear Reward Penalty</li>
<li>f: Linear Reward Inaction</li>
<li>f: UBC</li>
<li>g: BayesianUCB</li>
<li>h: Thompson Sampling</li>
<li>i: Neural Network</li>
<li>j: Non Stationary</li>

In [39]:
# import lib
import numpy as np
import matplotlib.pyplot as plt
import scipy,time,sys
np.random.seed(5678)
np.set_printoptions(3)

In [61]:
# setting the ground truth
num_bandit = 12
num_ep  = 200
num_iter= 2000
gt_prob = np.random.uniform(0,1,num_bandit)
optimal_choice = np.argmax(gt_prob)
print(gt_prob)
print('Best Choice: ',optimal_choice,gt_prob[optimal_choice])

[0.77  0.29  0.795 0.553 0.671 0.403 0.445 0.47  0.225 0.195 0.567 0.347]
Best Choice:  2 0.7947818672937569


In [62]:
# vectorized
a_expect = np.zeros((num_ep,num_bandit))
                    
for eps in range(num_ep):
    temp_expect = np.zeros(num_bandit)
    temp_choice = np.zeros(num_bandit)
                    
    for iter in range(num_iter//10):
        temp_choice    = temp_choice + 1
        current_reward = np.random.uniform(0,1) < gt_prob
        temp_expect    = temp_expect + (1/(temp_choice+1)) * (current_reward - temp_expect)
    a_expect[eps,:] = temp_expect
                    
print('Ground Truth')
print(gt_prob)
print('Expected ')
print(a_expect.mean(0))

Ground Truth
[0.77  0.29  0.795 0.553 0.671 0.403 0.445 0.47  0.225 0.195 0.567 0.347]
Expected 
[0.762 0.285 0.788 0.546 0.664 0.397 0.44  0.464 0.221 0.193 0.559 0.342]


In [63]:
# greedy
b_pull_count   = np.zeros((num_ep,num_bandit))
b_estimation   = np.zeros((num_ep,num_bandit))
b_reward       = np.zeros((num_ep,num_iter))
b_optimal_pull = np.zeros((num_ep,num_iter))
                    
for eps in range(num_ep):
    temp_pull_count   = np.zeros(num_bandit)
    temp_estimation   = np.zeros(num_bandit)
    temp_reward       = np.zeros(num_iter)
    temp_optimal_pull = np.zeros(num_iter)
                    
    for iter in range(num_iter):
        
        # select bandit / get reward /increase count / update estimate
        current_choice = np.argmax(temp_expect)
        current_reward = 1 if np.random.uniform(0,1) < gt_prob[current_choice] else 0
        temp_pull_count[current_choice] = temp_pull_count[current_choice] + 1
        temp_estimation[current_choice] = temp_estimation[current_choice] + (1/(temp_pull_count[current_choice]+1)) * (current_reward-temp_estimation[current_choice])
        
        # update reward and optimal choice
        temp_reward[iter] = temp_reward[iter] + current_reward
        temp_optimal_pull[iter] = 1 if current_choice == optimal_choice else 0
        
    b_pull_count[eps,:]   = temp_pull_count
    b_estimation[eps,:]   = temp_estimation
    b_reward[eps,:]       = temp_reward
    b_optimal_pull[eps,:] = temp_optimal_pull
        
print('Ground Truth')
print(gt_prob)
print('Expected ')
print(b_estimation.mean(0))

Ground Truth
[0.77  0.29  0.795 0.553 0.671 0.403 0.445 0.47  0.225 0.195 0.567 0.347]
Expected 
[0.    0.    0.795 0.    0.    0.    0.    0.    0.    0.    0.    0.   ]


In [68]:
# e greedy 
epsilon = np.random.uniform(0,1)
c_pull_count   = np.zeros((num_ep,num_bandit))
c_estimation   = np.zeros((num_ep,num_bandit))
c_reward       = np.zeros((num_ep,num_iter))
c_optimal_pull = np.zeros((num_ep,num_iter))
                    
for eps in range(num_ep):
    temp_pull_count   = np.zeros(num_bandit)
    temp_estimation   = np.zeros(num_bandit)
    temp_reward       = np.zeros(num_iter)
    temp_optimal_pull = np.zeros(num_iter)
                    
    for iter in range(num_iter):
        
        # select bandit / get reward /increase count / update estimate
        current_choice = np.argmax(temp_expect) if epsilon < np.random.uniform(0,1) else np.random.choice(np.arange(num_bandit))
        current_reward = 1 if np.random.uniform(0,1) < gt_prob[current_choice] else 0
        temp_pull_count[current_choice] = temp_pull_count[current_choice] + 1
        temp_estimation[current_choice] = temp_estimation[current_choice] + (1/(temp_pull_count[current_choice]+1)) * (current_reward-temp_estimation[current_choice])
        
        # update reward and optimal choice
        temp_reward[iter] = temp_reward[iter] + current_reward
        temp_optimal_pull[iter] = 1 if current_choice == optimal_choice else 0
        
    c_pull_count[eps,:]   = temp_pull_count
    c_estimation[eps,:]   = temp_estimation
    c_reward[eps,:]       = temp_reward
    c_optimal_pull[eps,:] = temp_optimal_pull
        
print('Ground Truth')
print(gt_prob)
print('Expected ')
print(c_estimation.mean(0))

Ground Truth
[0.77  0.29  0.795 0.553 0.671 0.403 0.445 0.47  0.225 0.195 0.567 0.347]
Expected 
[0.715 0.262 0.795 0.504 0.635 0.37  0.426 0.437 0.221 0.176 0.523 0.326]


In [71]:
# decy e greedy 
epsilon = 1.0
d_pull_count   = np.zeros((num_ep,num_bandit))
d_estimation   = np.zeros((num_ep,num_bandit))
d_reward       = np.zeros((num_ep,num_iter))
d_optimal_pull = np.zeros((num_ep,num_iter))
                    
for eps in range(num_ep):
    temp_pull_count   = np.zeros(num_bandit)
    temp_estimation   = np.zeros(num_bandit)
    temp_reward       = np.zeros(num_iter)
    temp_optimal_pull = np.zeros(num_iter)
                    
    for iter in range(num_iter):
        
        # select bandit / get reward /increase count / update estimate
        current_choice = np.argmax(temp_expect) if epsilon < np.random.uniform(0,1) else np.random.choice(np.arange(num_bandit))
        current_reward = 1 if np.random.uniform(0,1) < gt_prob[current_choice] else 0
        temp_pull_count[current_choice] = temp_pull_count[current_choice] + 1
        temp_estimation[current_choice] = temp_estimation[current_choice] + (1/(temp_pull_count[current_choice]+1)) * (current_reward-temp_estimation[current_choice])
        
        # update reward and optimal choice
        temp_reward[iter] = temp_reward[iter] + current_reward
        temp_optimal_pull[iter] = 1 if current_choice == optimal_choice else 0
        
        # decay the eps
        epsilon = 0.999 * epsilon
        
    d_pull_count[eps,:]   = temp_pull_count
    d_estimation[eps,:]   = temp_estimation
    d_reward[eps,:]       = temp_reward
    d_optimal_pull[eps,:] = temp_optimal_pull
        
print('Ground Truth')
print(gt_prob)
print('Expected ')
print(c_estimation.mean(0))

0.979835167579698
0.9788553324121183
0.9778764770797062
0.9768986006026265
0.9759217020020238
0.9749457803000218
0.9739708345197218
0.972996863685202
0.9720238668215168
0.9710518429546953
0.9700807911117406
0.9691107103206289
0.9681415996103083
0.967173458010698
0.9662062845526873
0.9652400782681346
0.9642748381898665
0.9633105633516766
0.962347252788325
0.9613849055355367
0.9604235206300011
0.9594630971093712
0.9585036340122618
0.9575451303782495
0.9565875852478712
0.9556309976626234
0.9546753666649608
0.9537206912982958
0.9527669706069976
0.9518142036363906
0.9508623894327541
0.9499115270433214
0.9489616155162781
0.9480126539007618
0.9470646412468611
0.9461175766056142
0.9451714590290087
0.9442262875699796
0.9432820612824097
0.9423387792211273
0.9413964404419062
0.9404550440014643
0.9395145889574629
0.9385750743685054
0.937636499294137
0.9366988627948428
0.935762163932048
0.9348264017681159
0.9338915753663478
0.9329576837909814
0.9320247261071904
0.9310927013810832
0.9301616086797022

0.22154835340667248
0.2213268050532658
0.22110547824821253
0.2208843727699643
0.22066348839719435
0.22044282490879716
0.22022238208388836
0.22000215970180448
0.21978215754210267
0.21956237538456058
0.21934281300917602
0.21912347019616685
0.21890434672597067
0.2186854423792447
0.21846675693686546
0.21824829017992858
0.21803004188974864
0.2178120118478589
0.21759419983601103
0.217376605636175
0.21715922903053883
0.2169420698015083
0.2167251277317068
0.21650840260397508
0.2162918942013711
0.21607560230716974
0.21585952670486258
0.21564366717815772
0.21542802351097956
0.21521259548746857
0.21499738289198111
0.21478238550908912
0.21456760312358003
0.21435303552045645
0.214138682484936
0.21392454380245107
0.21371061925864862
0.21349690863938997
0.21328341173075058
0.21307012831901984
0.2128570581907008
0.2126442011325101
0.21243155693137758
0.2122191253744462
0.21200690624907176
0.2117948993428227
0.21158310444347989
0.21137152133903642
0.21116014981769737
0.21094898966787967
0.2107380406782

0.05699490699016607
0.056937912083175904
0.05688097417109273
0.05682409319692164
0.05676726910372472
0.056710501834620994
0.05665379133278637
0.05659713754145358
0.056540540403912126
0.056483999863508216
0.05642751586364471
0.05637108834778106
0.05631471725943328
0.056258402542173847
0.05620214413963167
0.05614594199549204
0.05608979605349655
0.056033706257443056
0.05597767255118561
0.05592169487863442
0.05586577318375579
0.055809907410572034
0.05575409750316146
0.055698343405658296
0.05564264506225264
0.05558700241719038
0.055531415414773196
0.05547588399935842
0.05542040811535906
0.0553649877072437
0.055309622719536454
0.055254313096816915
0.0551990587837201
0.05514385972493638
0.05508871586521144
0.05503362714934623
0.05497859352219688
0.05492361492867469
0.05486869131374601
0.054813822622432264
0.054759008799809834
0.054704249791010025
0.054649545541219015
0.054594895995677795
0.054540301099682116
0.05448576079858243
0.05443127503778385
0.05437684376274606
0.05432246691898331
0.054

0.01818124433797811
0.01816306309364013
0.01814490003054649
0.018126755130515943
0.018108628375385426
0.01809051974701004
0.01807242922726303
0.018054356798035767
0.01803630244123773
0.018018266138796493
0.018000247872657697
0.01798224762478504
0.017964265377160255
0.017946301111783095
0.017928354810671312
0.01791042645586064
0.01789251602940478
0.017874623513375376
0.017856748889862002
0.01783889214097214
0.017821053248831167
0.017803232195582337
0.017785428963386755
0.017767643534423366
0.017749875890888944
0.017732126014998056
0.01771439388898306
0.017696679495094077
0.017678982815598984
0.017661303832783384
0.0176436425289506
0.017625998886421648
0.017608372887535226
0.01759076451464769
0.017573173750133043
0.01755560057638291
0.01753804497580653
0.017520506930830723
0.017502986423899893
0.017485483437475993
0.017467997954038517
0.01745052995608448
0.017433079426128394
0.017415646346702264
0.017398230700355562
0.017380832469655206
0.01736345163718555
0.017346088185548364
0.01732874

0.00405374048515334
0.004049686744668187
0.004045637057923519
0.004041591420865596
0.00403754982944473
0.004033512279615285
0.00402947876733567
0.0040254492885683345
0.004021423839279766
0.0040174024154404865
0.004013385013025046
0.004009371628012021
0.004005362256384009
0.0040013568941276246
0.003997355537233497
0.003993358181696263
0.003989364823514567
0.003985375458691052
0.0039813900832323616
0.003977408693149129
0.00397343128445598
0.003969457853171524
0.003965488395318352
0.003961522906923034
0.0039575613840161115
0.003953603822632095
0.0039496502188094635
0.003945700568590654
0.003941754868022063
0.003937813113154041
0.003933875300040887
0.003929941424740846
0.003926011483316105
0.003922085471832789
0.003918163386360956
0.003914245222974595
0.00391033097775162
0.003906420646773869
0.0039025142261270946
0.0038986117119009676
0.003894713100189067
0.0038908183870888778
0.003886927568701789
0.003883040641133087
0.003879157600491954
0.003875278442891462
0.0038714031644485705
0.003867

...

3.2721381696015464e-42
3.2688660314319447e-42
3.265597165400513e-42
3.2623315682351124e-42
3.259069236666877e-42
3.25581016743021e-42
3.25255435726278e-42
3.249301802905518e-42
3.2460525011026124e-42
3.24280644860151e-42
3.239563642152908e-42
3.2363240785107555e-42
3.233087754432245e-42
3.229854666677813e-42
3.2266248120111347e-42
3.2233981871991237e-42
3.220174789011925e-42
3.216954614222913e-42
3.21373765960869e-42
3.210523921949081e-42
3.2073133980271316e-42
3.2041060846291047e-42
3.2009019785444756e-42
3.197701076565931e-42
3.194503375489365e-42
3.191308872113876e-42
3.188117563241762e-42
3.1849294456785205e-42
3.181744516232842e-42
3.178562771716609e-42
3.1753842089448926e-42
3.1722088247359476e-42
3.1690366159112115e-42
3.1658675792953004e-42
3.162701711716005e-42
3.159539010004289e-42
3.1563794709942844e-42
3.15322309152329e-42
3.150069868431767e-42
3.146919798563335e-42
3.1437728787647714e-42
3.1406291058860067e-42
3.1374884767801205e-42
3.1343509883033404e-42
3.131216637315037

7.302952647768935e-43
7.295649695121166e-43
7.288354045426045e-43
7.281065691380618e-43
7.273784625689237e-43
7.266510841063549e-43
7.259244330222484e-43
7.251985085892261e-43
7.24473310080637e-43
7.237488367705562e-43
7.230250879337856e-43
7.223020628458519e-43
7.21579760783006e-43
7.20858181022223e-43
7.201373228412008e-43
7.194171855183596e-43
7.186977683328413e-43
7.179790705645084e-43
7.172610914939439e-43
7.1654383040244994e-43
7.158272865720475e-43
7.151114592854754e-43
7.143963478261899e-43
7.1368195147836375e-43
7.129682695268854e-43
7.122553012573585e-43
7.115430459561011e-43
7.10831502910145e-43
7.101206714072348e-43
7.0941055073582754e-43
7.087011401850917e-43
7.079924390449066e-43
7.072844466058617e-43
7.0657716215925584e-43
7.058705849970966e-43
7.051647144120995e-43
7.044595496976874e-43
7.037550901479898e-43
7.030513350578418e-43
7.0234828372278396e-43
7.016459354390612e-43
7.0094428950362214e-43
7.002433452141185e-43
6.995431018689044e-43
6.988435587670355e-43
6.981447

1.6282865055379836e-43
1.6266582190324457e-43
1.6250315608134132e-43
1.6234065292525998e-43
1.6217831227233471e-43
1.6201613396006238e-43
1.6185411782610231e-43
1.616922637082762e-43
1.6153057144456792e-43
1.6136904087312334e-43
1.6120767183225022e-43
1.6104646416041797e-43
1.6088541769625755e-43
1.6072453227856128e-43
1.6056380774628272e-43
1.6040324393853643e-43
1.602428406945979e-43
1.600825978539033e-43
1.5992251525604938e-43
1.5976259274079334e-43
1.5960283014805254e-43
1.5944322731790448e-43
1.5928378409058658e-43
1.5912450030649599e-43
1.5896537580618948e-43
1.588064104303833e-43
1.5864760401995291e-43
1.5848895641593295e-43
1.5833046745951701e-43
1.581721369920575e-43
1.5801396485506544e-43
1.5785595089021038e-43
1.5769809493932018e-43
1.5754039684438087e-43
1.5738285644753648e-43
1.5722547359108894e-43
1.5706824811749785e-43
1.5691117986938035e-43
1.5675426868951097e-43
1.5659751442082146e-43
1.5644091690640064e-43
1.5628447598949425e-43
1.5612819151350476e-43
1.55972063321991

3.6341067004493443e-44
3.630472593748895e-44
3.626842121155146e-44
3.623215279033991e-44
3.6195920637549573e-44
3.6159724716912025e-44
3.6123564992195114e-44
3.6087441427202917e-44
3.605135398577571e-44
3.6015302631789935e-44
3.5979287329158143e-44
3.5943308041828983e-44
3.5907364733787154e-44
3.5871457369053366e-44
3.5835585911684314e-44
3.5799750325772627e-44
3.5763950575446856e-44
3.572818662487141e-44
3.569245843824654e-44
3.5656765979808295e-44
3.5621109213828487e-44
3.5585488104614657e-44
3.554990261651004e-44
3.551435271389353e-44
3.5478838361179634e-44
3.5443359522818453e-44
3.5407916163295634e-44
3.5372508247132336e-44
3.53371357388852e-44
3.5301798603146316e-44
3.526649680454317e-44
3.5231230307738624e-44
3.5195999077430887e-44
3.5160803078353454e-44
3.51256422752751e-44
3.509051663299983e-44
3.5055426116366827e-44
3.502037069025046e-44
3.4985350319560207e-44
3.4950364969240646e-44
3.4915414604271406e-44
3.4880499189667134e-44
3.4845618690477465e-44
3.4810773071786987e-44
3.4

KeyboardInterrupt: 

# Reference 
1. numpy.set_printoptions — NumPy v1.14 Manual. (2019). Docs.scipy.org. Retrieved 13 January 2019, from https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.set_printoptions.html